In [1]:
import re
import os
import yaml

In [2]:
def create_tswap_instance(ins_file, output_file, agent_ids, map_width):
    """offline unlabeled-MAPF instance -> yaml problem for Online TSWAP"""
    r_pos = re.compile(r"(\d+),(\d+),(\d+),(\d+)")
    yaml_dict = {}
    agent_num = 0
    with open(ins_file, "r") as f:
        for row in f:
            m = re.match(r_pos, row)
            if m:
                yaml_dict[agent_ids[agent_num]] = {
                    "v": str(int(m.group(1)) + int(m.group(2))*map_width),
                    "g": str(int(m.group(3)) + int(m.group(4))*map_width),
                }
                agent_num += 1
                
    with open(output_file, "w") as f:
        yaml.dump(yaml_dict, f)

In [3]:
def create_mcps_plan(result_file, output_file, agent_ids, map_width):
    """offline unlabeled-MAPF result -> yaml plan for MCPs"""
    r_sol = re.compile(r"\d+:(.+)")
    r_pos = re.compile(r"\((\d+),(\d+)\),")
    
    plans = []
    with open(result_file, "r") as f:
        for row in f:
            m = re.match(r_sol, row)
            if m:
                m = re.findall(r_pos, m.group(1))
                plans.append([])
                for pos in m:
                    x = int(pos[0])
                    y = int(pos[1])
                    plans[-1].append(str(map_width * y + x))

    visited_cnt = {}
    yaml_dict = {}
    for agent_id in agent_ids:
        yaml_dict[agent_id] = { 'plan': [], 'order': [], }
        if len(yaml_dict.keys()) == len(plans[0]):
            break

    for t, c in enumerate(plans):
        for i, node_id in enumerate(c):
            if node_id not in visited_cnt.keys():
                visited_cnt[node_id] = 0
            if t > 0 and plans[t-1][i] == node_id:
                continue
            yaml_dict[agent_ids[i]]['plan'].append(node_id)
            yaml_dict[agent_ids[i]]['order'].append(visited_cnt[node_id])
            visited_cnt[node_id] += 1

    with open(output_file, "w") as f:
        yaml.dump(yaml_dict, f)

In [4]:
def create_yaml_map_file(map_file, output_file, x_start, y_start, grid_size):
    """offline unlabeled-MAPF map -> yaml map for Online TSWAP"""
    r_map = re.compile(r"map")
    r_width = re.compile(r"width\s(\d+)")
    r_height = re.compile(r"height\s(\d+)")
    width = 0
    height = 0

    y = 0
    yaml_dict = {}
    node_exists = []
    
    flg_map = False
    with open(map_file, "r") as f:
        for row in f:
            if flg_map:
                for x, s in enumerate(row.strip()):
                    node_exists.append(s == '.')
                    if s != '.': continue
                    id = str(y * width + x)
                    yaml_dict[id] = {}
                    yaml_dict[id]['pos'] = {
                        'x': x * grid_size + x_start,
                        'y': y * grid_size + y_start
                    }
                    yaml_dict[id]['neigh'] = []
                y += 1
                continue
            m = re.match(r_width, row)
            if m:
                width = int(m.group(1))
                continue
            m = re.match(r_height, row)
            if m:
                height = int(m.group(1))
                continue
            flg_map = bool(re.match(r_map, row))
    
    # create neighbors
    for node_id in yaml_dict.keys():
        id = int(node_id)
        x = id % width
        y = id // width
        # left
        if x > 0 and node_exists[y*width+x-1]:
            yaml_dict[node_id]['neigh'].append(str(y*width+x-1))
        # right
        if x < width - 1 and node_exists[y*width+x+1]:
            yaml_dict[node_id]['neigh'].append(str(y*width+x+1))
        # up
        if y > 0 and node_exists[(y-1)*width+x]:
            yaml_dict[node_id]['neigh'].append(str((y-1)*width+x))
        # down
        if y < height - 1 and node_exists[(y+1)*width+x]:
            yaml_dict[node_id]['neigh'].append(str((y+1)*width+x))

    with open(output_file, "w") as f:
        yaml.dump(yaml_dict, f)
        
       
    return width, height

In [5]:
def convert_from_libMotionPlanning_to_offlineTswapFile(result_filename, output_filename):
    """planning result from Wolfgang's repo -> unlabeled-MAPF plan"""
    output_str = "solution=\n"
    with open(result_filename) as f:
        res = yaml.safe_load(f)
    makespan = res['statistics']['makespan']
    configs = [ ['']*len(res['schedule']) for _ in range(makespan+1) ]
    paths = [ ]
    for i, agent in enumerate(res['schedule'].keys()):
        traj = res['schedule'][agent]
        for p in traj:
            configs[p['t']][i] = f'({p["x"]},{p["y"]})'
        # format
        for t in range(len(traj), makespan + 1):
            configs[t][i] = f'({p["x"]},{p["y"]})'
    for t, c in enumerate(configs):
        output_str += f"{t}:{','.join(c)},\n"
            
    with open(output_filename, 'w') as f:
        f.write(output_str)

In [6]:
x_start = 50
y_start = 550
grid_size = 38
width = 8
agent_ids = [
    'aaaaaaaa',
    'bbbbbbbb',
    'cccccccc',
    'dddddddd',
    'eeeeeeee',
    'ffffffff',
    'gggggggg',
    'hhhhhhhh',
]

In [ ]:
# create map file

map_file = "xxxxxxxx.map"
map_output_file = "xxxxxxxx.yaml"
create_yaml_map_file(map_file, map_output_file, x_start, y_start, grid_size)

In [ ]:
# create Online TSWAP problem file

from_file = "xxxxxxxx.txt"
to_file = "xxxxxxxx.yaml"
create_tswap_instance(from_file, to_file, agent_ids, width)

In [ ]:
# create offline plans for MCPs (from unlabeled-MAPF simulator)

filename_from = "xxxxxxxx.txt"
filename_to = "xxxxxxxx.yaml"
create_mcps_plan(filename_from, filename_to, agent_ids, width)

In [ ]:
# from libMotionPlanning -> unlabeled-MAPF-format -> offline plans for MCPs

filename_from = os.path.join(dirname, f"toio-{i}_origin.yaml")
filename_unlabeled = os.path.join(dirname, f"toio-{i}_mapfIR_format.txt")
filename_to = "xxxxxxxx.yaml"

convert_from_libMotionPlanning_to_offlineTswapFile(filename_from, filename_unlabeled)
create_mcps_plan(filename_unlabeled, filename_to, agent_ids, width)